# Exploration #1 가위바위보 분류기

## 라이브러리 불러오기

In [1]:
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from tensorflow import keras

## 데이터 불러오기 + Resize 하기


In [6]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path_1 = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
image_dir_path_2 = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
image_dir_path_3 = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
image_dir_path_4 = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/scissor"
image_dir_path_5 = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/rock"
image_dir_path_6 = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/paper"


resize_images(image_dir_path_1)
resize_images(image_dir_path_2)
resize_images(image_dir_path_3)
resize_images(image_dir_path_4)
resize_images(image_dir_path_5)
resize_images(image_dir_path_6)


print('이미지 resize 완료!')


100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
이미지 resize 완료!


## Train Data

In [7]:

def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0
x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 3)  
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 3)

학습데이터(x_train)의 이미지 개수는 300 입니다.
x_train shape: (300, 28, 28, 3)
y_train shape: (300,)


## Test Data

In [8]:
# x_test, y_test를 만드는 방법은 x_train, y_train을 만드는 방법과 아주 유사합니다.
def load_data(ima_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=300 # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(ima_path+'/test/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(ima_path+'/test/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(ima_path+'/test/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("테스트데이터(x_test)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

test_dir = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_test, y_test)=load_data(test_dir)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

테스트데이터(x_test)의 이미지 개수는 300 입니다.
x_test shape: (300, 28, 28, 3)
y_test shape: (300,)


## Model / Accuracy

In [9]:
n_channel_1=32
n_channel_2=64
n_channel_3 = 128
n_dense=512

IMG_HEIGHT = 28
IMG_WIDTH = 28

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', 
                              input_shape=(IMG_HEIGHT,IMG_WIDTH,3)))
model.add(keras.layers.MaxPooling2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, kernel_regularizer=keras.regularizers.l2(0.001),activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

model.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])



callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

history = model.fit(x_train_reshaped, y_train, batch_size=100,epochs=10, 
                    verbose=1, validation_data=(x_test_reshaped, y_test),
                    callbacks =[callback])


test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 128)        

/opt/conda/lib/python3.9/site-packages/keras/backend.py:4906: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


3/3 [==============================] - 3s 92ms/step - loss: 1.3004 - accuracy: 0.3400 - val_loss: 1.2901 - val_accuracy: 0.3333
Epoch 2/10
3/3 [==============================] - 0s 15ms/step - loss: 1.2735 - accuracy: 0.3500 - val_loss: 1.2745 - val_accuracy: 0.3333
Epoch 3/10
3/3 [==============================] - 0s 15ms/step - loss: 1.2469 - accuracy: 0.5067 - val_loss: 1.2588 - val_accuracy: 0.3467
Epoch 4/10
3/3 [==============================] - 0s 15ms/step - loss: 1.2117 - accuracy: 0.6433 - val_loss: 1.2405 - val_accuracy: 0.3267
Epoch 5/10
3/3 [==============================] - 0s 16ms/step - loss: 1.1636 - accuracy: 0.8133 - val_loss: 1.2216 - val_accuracy: 0.3333
Epoch 6/10
3/3 [==============================] - 0s 15ms/step - loss: 1.1072 - accuracy: 0.7967 - val_loss: 1.2065 - val_accuracy: 0.3133
Epoch 7/10
3/3 [==============================] - 0s 15ms/step - loss: 0.9938 - accuracy: 0.7733 - val_loss: 1.2028 - val_accuracy: 0.4600
Epoch 8/10
3/3 [======================